# Libraries

In [3]:
import fastai

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

#export
from exp.nb_02 import *
import torch.nn.functional as F # all nn functions
import torch.nn as nn

ImportError: cannot import name 'datasets' from 'fastai' (/opt/conda/lib/python3.7/site-packages/fastai/__init__.py)

## Get Data (MNIST)

In [ ]:
x_train,y_train,x_valid,y_valid = get_data()
n,m = x_train.shape
c = y_train.max()+1
nh = 50
n, m, nh
type(n), type(x_train), n, m

## Model

In [ ]:
# deifinition

# model contains the layers and a forward function (__call__)
class Model(nn.Module): # basically same as nn.module
    
    def __init__(self, layers):
        super().__init__() # sets the set attribute to add the layers to the 'modules dictionary'
        #self.layers = layers
        #for i,l in enumerate(self.layers): self.add_module(f'layer_{i}', l)
        self.layers = nn.ModuleList(layers) # equivalent to the loop above ading modules in Pytorch
    def __call__(self, x):
        for l in self.layers: x = l(x)
        return x
    
#??nn.Module
# Layers

layers = [nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,10)]

# New model object

net1 = Model(layers)

net2 = nn.Sequential(nn.Linear(m,nh), nn.ReLU(), nn.Linear(nh,10)) # pytorch version

net1, net2, net1.parameters(), # parameters generator function in nn.module generates the parameters 
#net1._modules

#n = nn.Linear(m, nh)


In [ ]:
# Generator example

def genEx():
    
    x = [1, 2, 3, 4]
    
    for dx in x:
        yield dx
        
ex = genEx()
print(next(ex))
print(next(ex))

for dx in genEx():
    
    print("Genrating:" +  str(dx))
    
ff = {"a" : 1, "b" : 2}
ff.values()

In [ ]:
bs = 50
#p = range((n-1)//bs + 1)
#x = range((n - 1)//bs + 1)
#xx = [i for i in x]

type(n)

## Loss function

In [ ]:
pred = net1(x_train[0:50])
loss = F.cross_entropy(pred, y_train[0:50])
loss
# Loss function 
#F.cross_entropy(net1())

# Accurac

def accuracy(out, yb): return (torch.argmax(out, dim=1)==yb).float().mean()
#accuracyz = accuracy(f, trainDS.y[0:50])
#torch.argmax(pred, dim = 1) == y_train[0:50]
#pred.shape, F.shape, accuracyz

## Training loop

In [ ]:
# Basic loop 
epochs = 1 # number of epochs
bs = 64 # batch size
lr = 0.5 # learning rate

for dx in range(epochs):
    for bdx in range((n-1)//bs + 1):
        x = x_train[bdx*bs:bdx*bs + bs]
        y = y_train[bdx*bs:bdx*bs + bs]
        
        #loss = loss_func(model(x), y)
        loss = F.cross_entropy(net1(x), y) # forward to loss
        
        # backward loop
        loss.backward()
        with torch.no_grad():
            for ldx in net1.layers: # loop through layers
                if hasattr(ldx, 'weight'): # check for weight attributes
                    ldx.weight -= lr*ldx.weight.grad # update weights
                    ldx.bias -= lr*ldx.bias.grad # upate bias
                    ldx.weight.grad.zero_()
                    ldx.bias.grad.zero_()
            
#
#print("Accuray after 1 epoch = " + str(accuracy(F.cross_entropy(net1(x_train[0:50]), y_train[0:50]), y_train[0:50])))
                    
# Optimizer to refactor code (note only needed for loss(the backward loop, so only need to update layers 
# with parameters (i.e. not relu))loss

class Optm():
    
    def __init__(self, params , lr = 0.5):
        self.params, self.lr = list(params), lr
        
    def step(self):
        for ldx in self.params():
            ldx -= lr*ldx.grad.zeros_()
        
        
        
    #def zero_grad():
        
opt = Optm(net1.parameters())
for dx in net1.parameters():
    print(dx.shape)

b = F.cross_entropy(net1(x_train[0:bs]), y_train[0:bs])
b
pred = net1(x_train[0:bs])
pred.shape
#a = accuracy(pred, y_train[0:bs])
#accuracy = accuracy(pred, y_train[0:50])

net1._modules, net1.parameters()

## Fully refactored

In [ ]:
## Optimiser- allows stepping through layers and updating and zeroing gradients 
# nn module parameters function returns tensor of layers with values
## This is equivalent to optim.sdg from pytorch
class Optimizer():
    def __init__(self, params, lr=0.5): self.params,self.lr=list(params), lr
        
    def step(self): # step through each parameter updating the weights
        with torch.no_grad():
            for p in self.params: p -= p.grad * lr

    def zero_grad(self): # step through each parameter updating the gradients
        for p in self.params: p.grad.data.zero_()
            
opt = Optimizer(net1.parameters())

from torch import optim
optTorch = optim.SGD(net1.parameters(), 0.5)

opt.params, 
optTorch.param_groups


## 


In [ ]:
class genEx2():
    def __init__(self):
            self.x = [1, 2, 3, 4, 5]
    
    def gen(self):
        for ix in self.x:
            yield ix


g = genEx2()

for gx in g.gen():
    print(gx)
    
gg = genEx2()

class genRec():
    def __init__(self, p):
        self.p = p
        
    def update(self):
        for px in self.p:
            px+= 1
        
gr = genRec(gg.gen())
gr.update()

for gx in gr.p:
    print(gx)

In [ ]:
it1 = iter([1, 2, 3])
it2 = iter([5, 6, 7])

next(it1), next(it2), next(it2), next(it1)

while True:
    try:
        print(next(it1))
    except StopIteration:
        break


## Create our own classes

In [ ]:
# Dataset - contains the x and y data. The __len__ method returns the length of the dataset 
# and the __getitem__ returns the x and y
class Dataset():
    def __init__(self,x, y):
        self.x, self.y = x, y
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, i):
        return self.x[i], self.y[i]


# Dataloader - uses a coroutine to request data from the dataset
# Takes a dataset object and a batch size as input - this data loader allows sequenctial loading

class Dataloader():
    def __init__(self, ds, bs = 64):
        self.ds = ds
        self.bs = bs
    def __iter__(self):
        for i in range(0, len(self.ds), self.bs): yield self.ds[i:i+self.bs]
    def __len__(self): return len(self.ds)//self.bs
            

# Data loader and related classes for optional random sampling. It just gives random samples of values from the 
#dataset

class Sampler():
    def __init__(self, ds, bs, shuffle=False):
        self.n,self.bs,self.shuffle = len(ds),bs,shuffle
        
    def __iter__(self):
        self.idxs = torch.randperm(self.n) if self.shuffle else torch.arange(self.n)
        for i in range(0, self.n, self.bs): yield self.idxs[i:i+self.bs]
            
# This class is different from Jeremys as is creates its own internal Sampler - 
#Jeremys allows the sampler to be passed in. This is probably more flexible
class DataloaderShuffle():
    def __init__(self, ds, bs, shuffle = False):
        self.ds = ds
        self.bs = bs
        self.Smplr = (Sampler(ds, bs, shuffle))
        
    def __iter__(self):
        for dx in self.Smplr:
            #print(dx)
            yield self.ds[dx] # this returns a tuple with index 0 containing the image and index 1 containing the
            # training set. Jeremey uses a collate function to unpack the tuple.This is more flexible can add padingeetc
    def __len__(self): return len(self.ds)//self.bs
# Define the model class =s=================================================================================== #
# pytorch type model, inheriting from nn.module The __call__ function is the forward pass
# nn layers can be passed to the model and registered using the nn.ModuleList function
# 
class nnModel(nn.Module):  
    def __init__(self, layers):
        super().__init__()
        self.layers = nn.ModuleList(layers)
        
    def __call__(self, x):
        for l in self.layers: x = l(x)
        return x
        
        
## Define the optimizer class ================================================================================ #
class Optimizer():
    def __init__(self, params, lr = 0.5):
        self.params, self.lr = list(params), lr
        
    def step(self): # steps through the parameters on the backward pass and updates them
        with torch.no_grad():
            for dx in self.params:
                dx -= dx.grad*lr
        
    def zero_grad(self): # zeros the gradients
        for p in self.params: p.grad.data.zero_()

## Define a minimum cross entropy loss function
#class MinCrossEntropy loss:

# Training loop 
def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        # Handle batchnorm / dropout
        model.train()
#         print(model.training)
        for xb,yb in train_dl:
            loss = loss_func(model(xb), yb)
            loss.backward()
            opt.step()
            opt.zero_grad()
            #print("In trainign" + str(epx))
        print("Training ac, epoch: " + str( epoch ) + " loss: " +str(loss))
        model.eval()
#         print(model.training)
        with torch.no_grad():
            tot_loss,tot_acc = 0.,0.
            for xb,yb in valid_dl:
                pred = model(xb)
                tot_loss += loss_func(pred, yb)
                tot_acc  += accuracy (pred,yb)
        nv = len(valid_dl)
        print(epoch, tot_loss/nv, tot_acc/nv, 180)
    return tot_loss/nv, tot_acc/nv

## Running

In [ ]:
# Test sequential data loader
ds_train = Dataset(x_train[0:29], y_train[0:29])

ld = Dataloader(ds_train, 3)
ldIt = iter(ld)
sx, sy = next((ldIt))
sx2, sy2 = next((ldIt))
sx3,sy3 = next(ldIt)
assert sy.shape == (3,)
sy, sy2, sy3 


# Sampler test ================================================================================================== #
smp = iter(Sampler(ds_train, 3, True))
smp1 = next(smp)
smp2 = next(smp)
smp1, smp2

smpF = iter(Sampler(ds_train, 3, True))
#for dx in smpF:
 #   print(dx)
#smp1F = next(smpF)
#smp2F = next(smpF)
#smp1F, smp2F

# Data loader with shuffler

#ld2 = DataloaderShuffle(ds_train, 3, True)
#ld2It = iter(ld2)
#ds, dt= next(ld2It)
#sy, y
#plt.imshow(sxx[0].view(28,28))
#syy[0]

#for ex in ld2It:
#    print(ex)

epochs = 5
bs = 50

## Set nn model
layers = [nn.Linear(m, nh), nn.ReLU(), nn.Linear(nh, 10)]
model1 = nnModel(layers)
    
## Set optizer class
optim = Optimizer(model1.parameters())

## Set dataset and data loader
trainDS = Dataset(x_train, y_train)
trainDLdr = DataloaderShuffle(trainDS, bs, True)

validDS = Dataset(x_valid, y_valid)
validDldr = DataloaderShuffle(validDS, bs, True)

ac1 = accuracy(model1(trainDS.x[0:1000]), trainDS.y[0:1000])
ac0 = accuracy(model1(validDS.x[0:1000]), trainDS.y[0:1000])
## Training loop 

#for epx in range(epochs):
 #   model1.train()
  #  for x,y in trainDLdr:
        #print(x.shape,y)
   #     pred = model1(x)
    #    loss = F.cross_entropy(pred, y)
     #   loss.backward()
      #  optim.step()
       # optim.zeroGrad()
    
    ## check accuracy on validation set
  #  model1.eval()
   # with torch.no_grad():
    #    tot_loss,tot_acc = 0.,0.
     #   print(epx)
      #  for xb,yb in validDldr:
#             print(epx)
#             pred = model1(xb)
#             tot_loss += F.cross_entropy(pred, yb)
#             tot_acc  += accuracy(pred,yb)
#             #print(F.cross_entropy(pred, yb))
#             #print("total loss =", str(tot_loss))
#     nv = len(validDS)
#     print(epx, tot_loss/nv, tot_acc/nv)
    #return tot_loss/nv, tot_acc/nv
fit(epochs, model1, F.cross_entropy, optim, trainDLdr, validDldr)
        
#, ds_train.y, y_train[0:28], plt.imshow(ds_train.x[3].view(28,28))
ac2 = accuracy(model1(trainDS.x[0:1000]), trainDS.y[0:1000])
ac3 = accuracy(model1(validDS.x[0:1000]), validDS.y[0:1000])
#ac1, ac0, ac2, ac3

x = model1(validDS.x[0:10])
y = validDS.y[0:10]
acc = accuracy(x,y)

ac1, ac0, ac2, ac3
#len(validDS)

## Pytorch equivalent

In [ ]:
class rndprm():
    def __init__(self):
        self.x = torch.randperm(5)
    def __iter__(self):
        
        for dx in range(5):
            yield self.x[dx]
            
x = iter(rndprm())
for dx in x:
    print(dx)
    
print("Going again")

for dx in x:
    print(dx)

In [ ]:
d = torch.randperm(20)
d[0:2]
x = (1, 2)
a, b = zip(*x)